In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df = pd.read_excel('./files/서울 지하철역 마스터시트.xlsx')
df = df.fillna(0)
df.describe()

,퇴근시간 하차인원,신용판매금액,문화예술관람,공연장수 대비 예매건수,역위도,역경도,근처 공연장수,공연건수,예매건수
count,234.00,234.00,234.00,234.00,234.00,234.00,234.00,234.00,234.00
mean,2855032.06,1566.43,5.76,54114.95,37.55,127.01,29.29,262.87,432911.59
std,2486569.98,345.65,2.25,92803.96,0.05,0.08,44.59,380.78,724393.43
min,0.00,1092.00,1.60,0.00,37.46,126.80,0.00,0.00,0.00
25%,1284361.25,1320.00,4.10,2451.71,37.51,126.95,4.00,25.50,9640.75
50%,2090592.50,1467.00,5.30,14962.43,37.55,127.02,12.00,92.00,139401.00
75%,3296192.50,1631.25,7.40,47092.55,37.57,127.07,30.00,317.50,405154.25
max,16652081.00,2336.00,10.80,365279.20,37.69,127.17,167.00,1464.00,2845374.00


In [5]:
def get_score(level, data):
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue 
    return score 

In [2]:
get_score([10,50,100],[5,15,70,120])

[1, 2, 3, 4]

In [6]:
def get_grade(df, num_class, tick_point, col_map):
    
    ##### 필요모듈 체크
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing

    ##### 파라미터 체크
    if not isinstance(df, pd.DataFrame): ## 데이터는 pd.DataFrame이어야 함.
        print('데이터는 pandas.DataFrame 객체여야 합니다.')
        return

    if isinstance(tick_point, dict) == False or isinstance(col_map, dict) == False: ## tick_point와 col_map은 모두 딕셔너리
        print(f'tick_point와 col_map은 모두 딕셔너리여야합니다.')
        return

    if not set(col_map.values()) == set(df[col_map.values()].columns):
        print('데이터 칼럼을 다시 확인해주세요')

    for k, v in tick_point.items():
        if isinstance(v, str):
            if not v in ['quantile','min_max']:
                print(f'{k}의 값은 "quantile" 또는 "min_max"중에 하나여야 합니다.')
                return
        elif isinstance(v,list) or isinstance(v,tuple):
            if len(v) != num_class[k]-1:
                print(f'{k}에 대응하는 리스트(튜플)의 원소는 {num_class[k]-1}개여야 합니다.')
                return
            
    for k, v in tick_point.items():
        scale = preprocessing.StandardScaler() ## 데이터 표준화
        temp_data = np.array(df[col_map[k]]) ## 데이터를 Numpy 배열로 변환
        temp_data = temp_data.reshape((-1,1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
        temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
        temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환

        if isinstance(v,str):
            if v == 'quantile':    
                ## 분위수 벡터
                quantiles_level = np.linspace(0,1,num_class[k]+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data,ql)) ## 분위수를 계산하고 리스트에 삽입

            else: ## min_max인 경우      
                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data),np.max(temp_data),num_class[k]+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점

        else: ## 직접 구분값을 넣어주는 경우
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = col_map[k]+'_'+k ## 점수값을 담는 변수의 이름

        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [7]:
tick_point={'score_1':'quantile','score_2':'quantile',
            'score_3':'quantile', 'score_4':'quantile'}

col_map={'score_1':'퇴근시간 하차인원','score_2':'공연장수 대비 예매건수'
         ,'score_3':'신용판매금액','score_4':'문화예술관람'}

num_class= {'score_1':5,'score_2':5,'score_3':3,'score_4':3}


In [8]:
result = get_grade(df=df, num_class=num_class, 
                   tick_point = tick_point, col_map = col_map)
result[['퇴근시간 하차인원_score_1','공연장수 대비 예매건수_score_2',
        '신용판매금액_score_3','문화예술관람_score_4']]

,퇴근시간 하차인원_score_1,공연장수 대비 예매건수_score_2,신용판매금액_score_3,문화예술관람_score_4
0,4,2,3,1
1,3,2,1,3
2,5,4,3,3
3,1,4,3,3
4,4,2,2,1
...,...,...,...,...
229,4,1,1,3
230,5,1,2,2
231,3,2,2,2
232,1,5,1,1


In [12]:
result['문화점수'] = (result['공연장수 대비 예매건수_score_2']+result['문화예술관람_score_4'])
result['인구점수'] = (result['퇴근시간 하차인원_score_1']+result['신용판매금액_score_3'])         
result['최종점수'] = (result['문화점수']+ result['인구점수'])/2
result_df = result.sort_values(by='최종점수', ascending=False)
result_df = result_df.reset_index(drop=True)
result_df[['역사명','문화점수','인구점수','최종점수']]   #최종 점수는 총 8점만점

,역사명,문화점수,인구점수,최종점수
0,강남,7,8,7.50
1,신사,8,7,7.50
2,목동,7,8,7.50
3,오목교,7,8,7.50
4,압구정,8,7,7.50
...,...,...,...,...
229,개화산,3,3,3.00
230,신내,4,2,3.00
231,마곡,3,3,3.00
232,여의나루,4,2,3.00


In [11]:
final_df = result_df[result_df['최종점수']>=6]  ## 최종 점수가 6점 이상인 지하철역 추출
final_df

,역사명,자치구,역위도,역경도,역사도로명주소,근처 공연장수,공연건수,예매건수,퇴근시간 하차인원(17시~1시),신용판매금액,문화예술관람,한 공연당 예매건수,퇴근시간 하차인원(17시~1시)_score_1,한 공연당 예매건수_score_2,신용판매금액_score_3,문화예술관람_score_4,문화점수,인구점수,최종점수
0,강남,강남구,37.49720,127.0281,서울특별시 강남구 강남대로 지하 396 (역삼동),32,717,729016,8523024,2326,9.0,32536.280,5,4,3,3,7,8,7.5
1,신사,강남구,37.51619,127.0201,서울특별시 강남구 도산대로 지하 102 (신사동),27,141,689421,3000049,2326,9.0,132016.800,4,5,3,3,8,7,7.5
2,목동,양천구,37.52587,126.8646,서울특별시 양천구 오목로 지하245 (목동),6,22,127899,4354611,1635,6.9,34881.550,5,4,3,3,7,8,7.5
3,오목교,양천구,37.52437,126.8751,서울특별시 양천구 오목로 지하342 (목동),12,40,135918,3930592,1635,6.9,40775.400,5,4,3,3,7,8,7.5
4,압구정,강남구,37.52669,127.0284,서울특별시 강남구 압구정로 지하 172 (신사동),30,155,476649,2699955,2326,9.0,92254.650,4,5,3,3,8,7,7.5
5,역삼,강남구,37.50054,127.0364,서울특별시 강남구 테헤란로 지하 156 (역삼동),37,188,286744,2876699,2326,9.0,56433.660,4,4,3,3,7,7,7.0
6,왕십리,성동구,37.56153,127.0375,서울특별시 성동구 왕십리로 지하300(행당동),12,51,259636,3585752,1599,8.4,61090.820,4,5,2,3,8,6,7.0
7,구의,광진구,37.53692,127.0857,서울특별시 광진구 아차산로 384-1 (구의동),10,37,95648,5474505,1382,8.3,25850.810,5,4,2,3,7,7,7.0
8,성신여대입구,성북구,37.59274,127.0163,서울특별시 성북구 동소문로 지하 102 (동선동4가),136,1024,2076858,3996304,1339,7.4,275832.700,5,5,1,3,8,6,7.0
9,구파발,은평구,37.63665,126.9188,서울특별시 은평구 진관2로 지하 15-25 (진관동),18,79,241770,4232637,1445,9.3,55086.840,5,4,2,3,7,7,7.0


In [11]:
final_df['최종점수'].value_counts()

6.0    23
7.0    12
6.5    10
7.5     5
Name: 최종점수, dtype: int64

In [14]:
final_df.value_counts('자치구')

자치구
강남구    12
광진구     7
마포구     7
성동구     7
서초구     4
양천구     4
성북구     3
송파구     2
은평구     2
용산구     1
중구      1
dtype: int64

In [12]:
final_df.to_excel('./files./키오스크 설치 후보 상위 50개 지하철역.xlsx', index = False)

In [13]:
result_df.to_excel('./files./전체 지하철역 점수.xlsx', index = False)